# Predict new flight positions

## 1. Load the saved model

In [11]:
# Import necessary libraries
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import math

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Context size of our custom model
context_size = 2048

class PositionEmbedding(torch.nn.Module):
    """Token and positioning embedding layer for a sequence."""
    def __init__(self):
        """Init variables and layers."""
        super().__init__()
        
        self.position_emb = torch.nn.Embedding(num_embeddings=context_size, embedding_dim=6)
    
    def forward(self, x):
        """Forward Pass."""
        len_input = x.size()[1]
        positions = torch.arange(start=0, end=len_input, step=1).to(device)
        position_embedding = self.position_emb(positions)
        return x + position_embedding

def create_attention_mask(key_length, query_length, dtype):
    """
    Create a Casual Mask for
    the multi head attention layer.
    """
    i = torch.arange(query_length)[:, None]
    j = torch.arange(key_length)
    mask = i >= j - key_length + query_length
    mask = torch.logical_not(mask)
    mask = mask.to(dtype)
    return mask

class TransformerBlock(torch.nn.Module):
    """Transformer Block Layer."""
    def __init__(self, num_heads, embed_dim, ff_dim, mask_function, dropout_rate=0.1):
        """Init variables and layers."""
        super().__init__()
        self.attn = torch.nn.MultiheadAttention(
          embed_dim=embed_dim,
          num_heads=num_heads,
          batch_first=True,
        )
        self.dropout_1 = torch.nn.Dropout(p=dropout_rate)
        self.layer_norm_1 = torch.nn.LayerNorm(
          normalized_shape=embed_dim, eps=1e-6
        )
        self.ffn_1 = torch.nn.Linear(
          in_features=embed_dim, out_features=ff_dim
        )
        self.ffn_2 = torch.nn.Linear(
          in_features=ff_dim, out_features=embed_dim
        )
        self.dropout_2 = torch.nn.Dropout(p=dropout_rate)
        self.layer_norm_2 = torch.nn.LayerNorm(
          normalized_shape=embed_dim, eps=1e-6
        )
        self.mask_function = mask_function

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        """Forward Pass."""
        seq_len = inputs.size()[1]
        mask = self.mask_function(seq_len, seq_len, torch.bool).to(device)
        attention_output, _ = self.attn(
        query=inputs, key=inputs, value=inputs, attn_mask=mask
        )
        attention_output = self.dropout_1(attention_output)
        out1 = self.layer_norm_1(inputs + attention_output)
        ffn_1 = self.ffn_1(out1)
        ffn_2 = self.ffn_2(ffn_1)
        ffn_output = self.dropout_2(ffn_2)
        output = self.layer_norm_2(out1 + ffn_output)
        return output

class FlightModel(torch.nn.Module):
  def __init__(self, feed_forward_dim, num_heads):
    """Init Function."""
    super().__init__()
    self.embedding_layer = PositionEmbedding()
    self.transformer_layers = []
    for i in range(6):
        transformer = TransformerBlock(
          num_heads=num_heads,
          embed_dim=6,
          ff_dim=feed_forward_dim,
          mask_function=create_attention_mask,
        ).to(device)
        self.transformer_layers.append(transformer)
        
    self.output_layer = torch.nn.Linear(6, 6)

  def forward(self, input_tensor):
    """Forward Pass."""
    # Position embedding
    embedding = self.embedding_layer(input_tensor)
    # Transformer layers
    transformer_output = self.transformer_layers[0](embedding)
    for i in range(1, len(self.transformer_layers)):
        transformer_output = self.transformer_layers[i](transformer_output)
    # FC network
    output = self.output_layer(transformer_output)
    return output


model = FlightModel(24, 1).to(device)
model.load_state_dict(torch.load("./flight_prediction_model_exp.pt", weights_only=True))
model.eval()

cuda


FlightModel(
  (embedding_layer): PositionEmbedding(
    (position_emb): Embedding(2048, 6)
  )
  (output_layer): Linear(in_features=6, out_features=6, bias=True)
)

## 2. Generate new plane positions

In [12]:
plane_pos_1 = [-33.625946, -70.909181, 14550, 327.9, 22, -64]
plane_pos_2 = [-33.607635, -70.900381, 14225, 324.4, 21.9, -900]
plane_pos = [plane_pos_1, plane_pos_2]
pos_input = torch.tensor(plane_pos)
pos_input = pos_input.view(1, -1, 6)
pos_input = pos_input.to(device)
torch.set_printoptions(sci_mode=False)

with torch.no_grad():
    for i in range(10):  
        output = model(pos_input)
        output = torch.squeeze(output)
        output = output[-1]
        div_tensor = torch.tensor([90,  180, 10000, 1000, 360, 1000]).to(device)
        output = torch.mul(output, div_tensor)
        output = output.view(1, 1, 6)
        pos_input = torch.cat((pos_input, output), 1)

print(pos_input)


tensor([[[   -33.6259,    -70.9092,  14550.0000,    327.9000,     22.0000,
             -64.0000],
         [   -33.6076,    -70.9004,  14225.0000,    324.4000,     21.9000,
            -900.0000],
         [    36.2456,    -26.7269,   1513.5985,     70.9948,    163.8928,
            -101.4011],
         [    47.6427,    -74.3880,   2403.3401,     41.0355,    201.9553,
             -69.3190],
         [    34.0646,    -43.2149,   1161.7010,    102.0613,    159.6414,
             -42.8755],
         [    30.8297,    -12.7706,    977.0361,     92.5054,    148.9762,
             -93.7314],
         [    30.6272,      2.6675,    904.8125,     72.3756,    109.5661,
            -127.9924],
         [    33.3599,      9.7938,   1246.7458,     44.9890,    121.7768,
            -169.1501],
         [    32.1715,     -3.9269,   1164.5562,     72.0250,    140.9974,
            -127.6193],
         [    36.9437,    -40.6239,   1469.4241,     82.9010,    178.1634,
             -68.4417],
         [